# Параметры

In [ ]:
!pip install pyDOE

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from collections import OrderedDict #упорядоченный словарь
from pyDOE import lhs #функция, выбирающая значения для обучения на них нейросети
import time
np.random.seed(1234)
if torch.cuda.is_available(): #можно сменить среду выполнения на gpu и обучение будет происходить быстрее
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

### Настраиваемые параметры
exp_name="VT_1(10)"
source_exp_name="BL_1(10)"
alpha_value=0.20
###

if "BL" in exp_name:
  basic_learning=True
  weights_transfer=False
  values_transfer=False
if "WT" in exp_name:
  basic_learning=False
  weights_transfer=True
  values_transfer=False
if "VT" in exp_name:
  basic_learning=False
  weights_transfer=False
  values_transfer=True

try:
    os.mkdir((f"exp_{exp_name}"))
except:
    pass

stats_file = open(f"exp_{exp_name}/stats.txt","a")

In [ ]:
#параметры области:
x_0=-60
x_1=40
t_0=0
t_1=20
x_parts=1000 # разбиение по x
t_parts=200 # разбиение по t
#параметры начального условия:
k_param=1
w_param=0.88
x0_param=-30
th0_param=0
alpha0_param=alpha_value
#параметры уравнения:
alpha=alpha_value
beta=0

def q(x,t):
    mu=4*(k_param**2-w_param)
    k_exp = np.exp(np.sqrt(mu)*(x - 2*k_param*t - x0_param))
    f_com = ((mu*k_exp) / ((0.5*k_exp+1)**2 - k_exp*k_exp*alpha0_param*mu/3))**0.5
    u = f_com*np.cos(k_param*x - w_param*t + th0_param)
    v = f_com*np.sin(k_param*x - w_param*t + th0_param)
    u=np.nan_to_num(u) #нужно на больших областях, когда на краях области появляются Nan
    v=np.nan_to_num(v)
    return u,v

# PINN

In [ ]:
class SinActivation(torch.nn.Module): #кастомная функция активации - sin
    def __init__(self):
        super(SinActivation, self).__init__()
        return
    def forward(self, x):
        return torch.sin(x)

class DNN(torch.nn.Module): # нейросеть, в виде которой будет находиться решение
    def __init__(self, layers): #принимает на вход массив целых чисел
        super(DNN, self).__init__() #вызывает метод init(почему нельзя сделать это без super?)

        self.depth = len(layers) - 1
        self.activation = SinActivation #в качестве функции активации используется sin

        layer_list = list() #список с весами и функциями активации для каждого слоя
        for i in range(self.depth - 1):
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1])) #каждые два слоя образуют двудольный граф
            )
            layer_list.append(('activation_%d' % i, self.activation()))

        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1])) #нельзя сделать в цикле, потому что нет функции активации
        )

        layerDict = OrderedDict(layer_list) #сделали упорядоченный словарь, чтобы при использовании элементы выдавались в том порядке, в котором были добавлены

        # deploy layers
        self.layers = torch.nn.Sequential(layerDict) #задали архитектуру нейросети
    def forward(self, x):
        out = self.layers(x)
        return out

class PhysicsInformedNN(): # PINN
    def __init__(self, X_tl, u_tl, v_tl, X_uv, u, v, layers, alpha, beta):

        self.X_tl = X_tl
        self.u_tl = u_tl
        self.v_tl = v_tl
        self.X_uv = X_uv
        self.u = u
        self.v = v
        # данные для обучения
        if values_transfer:
            #для value_transfer: (x_tl, t_tl, u, v)
            idx = np.random.choice(self.X_tl.shape[0], N_vt, replace=False)
            self.actual_x_tl = torch.tensor(self.X_tl[idx, 0:1], requires_grad=True).float().to(device)
            self.actual_t_tl = torch.tensor(self.X_tl[idx, 1:2], requires_grad=True).float().to(device)
            self.actual_u_tl = torch.tensor(self.u_tl[idx, :]).float().to(device)
            self.actual_v_tl = torch.tensor(self.v_tl[idx, :]).float().to(device)
        #для начальных и граничных условий: (x_uv, t_uv, u, v)
        idx = np.random.choice(self.X_uv.shape[0], N_ib, replace=False)
        self.actual_x_uv = torch.tensor(self.X_uv[idx, 0:1], requires_grad=True).float().to(device)
        self.actual_t_uv = torch.tensor(self.X_uv[idx, 1:2], requires_grad=True).float().to(device)
        self.actual_u = torch.tensor(self.u[idx, :]).float().to(device)
        self.actual_v = torch.tensor(self.v[idx, :]).float().to(device)
        #для уравнения: (x_f, t_f, f=0)
        idx = np.random.choice(self.X_tl.shape[0], N_eq, replace=False)
        self.actual_x_f = torch.tensor(self.X_tl[idx, 0:1], requires_grad=True).float().to(device)
        self.actual_t_f = torch.tensor(self.X_tl[idx, 1:2], requires_grad=True).float().to(device)

        # числовые коэффициенты в уравнении
        self.alpha = alpha
        self.beta = beta

        # модель
        self.layers = layers
        self.dnn = DNN(layers).to(device)

        # оптимизатор - LBFGS, обучает с точностью до 1e-5 или пока разница в точности уменьшается больше, чем точность float(?)
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(),
            lr=0.00001,
            max_iter=50000,
            max_eval=50000,
            history_size=50,
            tolerance_grad=1e-6,
            tolerance_change=1.0 * np.finfo(float).eps,
            #line_search_fn="strong_wolfe"
        )

        self.adam = torch.optim.Adam(
          self.dnn.parameters(),
          lr=0.005,
          betas=(0.9, 0.999),
          eps=1e-08,
          weight_decay=0,
          amsgrad=False)

        self.iter = 0

    def net_uv(self, x, t): # вывод модели
        u = self.dnn(torch.cat([x, t], dim=1))[:,0:1]
        v = self.dnn(torch.cat([x, t], dim=1))[:,1:2]
        return u, v

    def net_f(self, x, t): #вывод  функции
        """ The pytorch autograd version of calculating residual """
        u, v = self.net_uv(x, t)

        u_t = torch.autograd.grad(
            u, t,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0] #производая по t
        u_x = torch.autograd.grad(
            u, x,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0] #производная по x
        u_xx = torch.autograd.grad(
            u_x, x,
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0] #вторая призводная по x

        v_t = torch.autograd.grad(
            v, t,
            grad_outputs=torch.ones_like(v),
            retain_graph=True,
            create_graph=True
        )[0] #производая по t
        v_x = torch.autograd.grad(
            v, x,
            grad_outputs=torch.ones_like(v),
            retain_graph=True,
            create_graph=True
        )[0] #производная по x
        v_xx = torch.autograd.grad(
            v_x, x,
            grad_outputs=torch.ones_like(v_x),
            retain_graph=True,
            create_graph=True
        )[0] #вторая призводная по x

        f_u = u*(u**2 + v**2)*(-self.alpha*(u**2 + v**2) + self.beta*(u**2 + v**2)**2 + 1) + u_xx - v_t #это и есть действительная и коплексная части исходного уравнения
        f_v = u_t + v*(u**2 + v**2)*(-self.alpha*(u**2 + v**2) + self.beta*(u**2 + v**2)**2 + 1) + v_xx #они получены путём подстановки q=u+i*v в него
        return f_u, f_v

    def loss_func(self): #функция потерь
        self.optimizer.zero_grad() #обнуляет градиенты
        if values_transfer and self.iter<=transfer_period: # учим удовлетворять значениям
          u_pred, v_pred = self.net_uv(self.actual_x_tl, self.actual_t_tl)
          loss_tl = torch.mean(((self.actual_u_tl - u_pred) ** 2 + (self.actual_v_tl - v_pred) ** 2)/2) #средний квадрат всех отклонений от известных значений
          loss = loss_tl
        else: # учим удовлетворять уравнению
          u_pred, v_pred = self.net_uv(self.actual_x_uv, self.actual_t_uv)
          f_u_pred, f_v_pred = self.net_f(self.actual_x_f, self.actual_t_f)
          loss_uv = torch.mean(((self.actual_u - u_pred) ** 2 + (self.actual_v - v_pred) ** 2)/2) #средний квадрат всех отклонений от начальных и граничных условий
          loss_f = torch.mean((f_u_pred ** 2 + f_v_pred ** 2)/2) #средний квадрат всех отклонений от условия
          loss = loss_uv + loss_f

        loss.backward()
        self.iter += 1
        if self.iter % 100 == 0: #каждые 100 итераций перегенерируем точки
            idx = np.random.choice(self.X_uv.shape[0], N_ib, replace=False)
            self.actual_x_uv = torch.tensor(self.X_uv[idx, 0:1], requires_grad=True).float().to(device)
            self.actual_t_uv = torch.tensor(self.X_uv[idx, 1:2], requires_grad=True).float().to(device)
            self.actual_u = torch.tensor(self.u[idx, :]).float().to(device)
            self.actual_v = torch.tensor(self.v[idx, :]).float().to(device)
            
            idx = np.random.choice(self.X_tl.shape[0], N_eq, replace=False)
            self.actual_x_f = torch.tensor(self.X_tl[idx, 0:1], requires_grad=True).float().to(device)
            self.actual_t_f = torch.tensor(self.X_tl[idx, 1:2], requires_grad=True).float().to(device)
            
            if values_transfer:
                idx = np.random.choice(self.X_tl.shape[0], N_vt, replace=False)
                self.actual_x_tl = torch.tensor(self.X_tl[idx, 0:1], requires_grad=True).float().to(device)
                self.actual_t_tl = torch.tensor(self.X_tl[idx, 1:2], requires_grad=True).float().to(device)
                self.actual_u_tl = torch.tensor(self.u_tl[idx, :]).float().to(device)
                self.actual_v_tl = torch.tensor(self.v_tl[idx, :]).float().to(device)

        if self.iter % 1000 == 0:
            if values_transfer and self.iter <= transfer_period:
              print('Iter %d, Loss: %.5e' % (self.iter, loss.item()))
            else:
              print('Iter %d, Loss: %.5e, Loss_uv: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_uv.item(), loss_f.item()))
            loss_array.append(loss.item()) #loss запоминаем для графика
            iter_array.append(self.iter)

        return loss

    def train(self): #обучение
        adam_iterations = 100000
        if values_transfer:
          adam_iterations += transfer_period
        print('training started')
        print('%d iterations of ADAM:' %adam_iterations)
        self.dnn.train()
        for i in range(adam_iterations): #во время тренировки производится 1000 шагов adam
            if i % 100 == 0: self.adam.param_groups[0]['lr'] = 0.99*self.adam.param_groups[0]['lr'] #экспоненциальное уменьшение шага каждые 100 шагов
            if values_transfer and i==transfer_period: self.adam.param_groups[0]['lr'] = 0.005
            self.adam.step(self.loss_func)
        #print('LBFGS:') #а дальше запускается lbfgs
        #self.optimizer.step(self.loss_func)
        print('Total iterations: %d + %d' %(adam_iterations, (self.iter-adam_iterations)))


    def predict(self, X): #вывод нейросети и функции на входных данных
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u, v = self.net_uv(x, t)
        f_u, f_v = self.net_f(x, t)
        u = u.detach().cpu().numpy()
        v = v.detach().cpu().numpy()
        f_u = f_u.detach().cpu().numpy()
        f_v = f_v.detach().cpu().numpy()
        return u, v, f_u, f_v

In [ ]:
if weights_transfer:
  raw_model = torch.load(f'model_{source_exp_name}.pth', map_location=device) #загружаем модель
if values_transfer:
  raw_data = pd.read_csv(f'data_{source_exp_name}.csv') #загружаем данные для обучения
  raw_t_0=t_0
  raw_t_1=t_1
  raw_x_0=x_0
  raw_x_1=x_1
  raw_t_parts=t_parts
  raw_x_parts=x_parts
  raw_X = np.array(raw_data['x']).reshape(raw_t_parts,raw_x_parts)
  raw_T = np.array(raw_data['t']).reshape(raw_t_parts,raw_x_parts)
  raw_U = np.array(raw_data['pred_u']).reshape(raw_t_parts,raw_x_parts)
  raw_V = np.array(raw_data['pred_v']).reshape(raw_t_parts,raw_x_parts)
  raw_Q_abs = np.array(raw_data['pred_h']).reshape(raw_t_parts,raw_x_parts)

In [ ]:
N_ib = 1000 # число точек, обучающих принимать граничные значения
N_eq = 50000 # число точек, обучающих удовлетворять уравнению
N_vt = 80000 # число точек, обучающих принимать значения предыдущей модели
layers = [2, 50, 50, 50, 2] # 2 входа, 2 выхода и 3 слоя по 50 нейронов
transfer_period = 20000
x = np.linspace(x_0, x_1, x_parts)
t = np.linspace(t_0, t_1, t_parts)
X, T = np.meshgrid(x, t)

Exact_q=q(X,T)[0] + 1j*q(X,T)[1]
Exact_u=np.real(Exact_q)
Exact_v=np.imag(Exact_q)
Exact_q_abs = np.abs(Exact_q)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact_u.flatten()[:,None]
v_star = Exact_v.flatten()[:,None]

# границы области
lb = X_star.min(0)
ub = X_star.max(0)

# для обучения берём только данные на границах области(граничные условия по сути)
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T)) #(x,t_0)
uu1 = Exact_u[0:1,:].T
vv1 = Exact_v[0:1,:].T
xx2 = np.hstack((X[:,0:1], T[:,0:1])) #(x_0,t)
uu2 = np.zeros((t_parts,1)) #граничные условия поставил 0 в соответствии с требованиями
vv2 = np.zeros((t_parts,1))
xx3 = np.hstack((X[:,-1:], T[:,-1:])) #(x_1,t)
uu3 = np.zeros((t_parts,1))
vv3 = np.zeros((t_parts,1))

X_uv_train = np.vstack([xx1, xx2, xx3]) #данные для тренировки в точках на границе
u_train = np.vstack([uu1, uu2, uu3])
v_train = np.vstack([vv1, vv2, vv3])
#X_f_train = lb + (ub-lb)*lhs(2, N_eq) #данные для тренировки в случайных точкам из области
#X_f_train = np.vstack((X_f_train, X_uv_train)) #добавим к ним ещё и точки на границе, там f тоже 0
X_f_train =  X_star #сети передадим все имеющиеся точки, в процессе обучения будет выбираться набор из N_eq

#idx = np.random.choice(X_uv_train.shape[0], N_ib, replace=False) #выберем из точек на границе только N_ib
#X_uv_train = X_uv_train[idx, :]
#u_train = u_train[idx,:]
#v_train = v_train[idx,:]

loss_array = [] #массив с loss в процессе обучения
iter_array = [] #итерации с шагом 100

In [ ]:
if values_transfer:
  Q_truth=q(raw_X,raw_T)[0]+1j*q(raw_X,raw_T)[1] #за правильные значения берутся значения функции при ТЕКУЩЕМ alpha
  U_truth=np.real(Q_truth)
  V_truth=np.imag(Q_truth)
  Q_abs_truth=np.abs(Q_truth)

  fig, axs = plt.subplots(3, 3, figsize=(21,10), dpi=300)

  for ax in axs.flat:
      ax.set(xlabel='$t$', ylabel='$x$')

  for ax in axs.flat:
      ax.label_outer()

  q_abs_min, q_abs_max = 0, np.abs(Q_abs_truth).max()
  c = axs[0,0].pcolormesh(raw_T, raw_X, Q_abs_truth, cmap='BuPu', vmin=q_abs_min, vmax=q_abs_max)
  axs[0,0].set_title('$|q_{truth}|(x,t)$')
  axs[0,0].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[0,0])

  u_min, u_max = -np.abs(U_truth).max(), np.abs(U_truth).max()
  c = axs[0,1].pcolormesh(raw_T, raw_X, U_truth, cmap='RdBu', vmin=u_min, vmax=u_max)
  axs[0,1].set_title('$u_{truth}(x,t)$')
  axs[0,1].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[0,1])

  c = axs[0,2].pcolormesh(raw_T, raw_X, V_truth, cmap='RdBu', vmin=u_min, vmax=u_max)
  axs[0,2].set_title('$v_{truth}(x,t)$')
  axs[0,2].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[0,2])


  #q_abs_min, q_abs_max = 0, np.abs(Q_abs_calc).max()
  c = axs[1,0].pcolormesh(raw_T, raw_X, raw_Q_abs, cmap='BuPu', vmin=q_abs_min, vmax=q_abs_max)
  axs[1,0].set_title('$|q_{pred}|(x,t)$')
  axs[1,0].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[1,0])

  #u_min, u_max = -np.abs(U_calc).max(), np.abs(U_calc).max()
  c = axs[1,1].pcolormesh(raw_T, raw_X, raw_U, cmap='RdBu', vmin=u_min, vmax=u_max)
  axs[1,1].set_title('$u_{pred}(x,t)$')
  axs[1,1].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[1,1])

  c = axs[1,2].pcolormesh(raw_T, raw_X, raw_V, cmap='RdBu', vmin=u_min, vmax=u_max)
  axs[1,2].set_title('$v_{pred}(x,t)$')
  axs[1,2].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[1,2])


  U_diff = np.abs(U_truth-raw_U)
  V_diff = np.abs(V_truth-raw_V)
  Q_abs_diff = np.abs(Q_abs_truth-raw_Q_abs)

  q_abs_min, q_abs_max = 0, np.abs(Q_abs_diff).max()
  c = axs[2,0].pcolormesh(raw_T, raw_X, Q_abs_diff, cmap='Reds', vmin=q_abs_min, vmax=q_abs_max)
  axs[2,0].set_title('$||q_{truth}|-|q_{pred}||(x,t)$')
  axs[2,0].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[2,0])

  u_min, u_max = U_diff.min(), U_diff.max()
  c = axs[2,1].pcolormesh(raw_T, raw_X, U_diff, cmap='Reds', vmin=u_min, vmax=u_max)
  axs[2,1].set_title('$|u_{truth}-u_{pred}|(x,t)$')
  axs[2,1].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[2,1])

  c = axs[2,2].pcolormesh(raw_T, raw_X, V_diff, cmap='Reds', vmin=u_min, vmax=u_max)
  axs[2,2].set_title('$|v_{truth}-v_{pred}|(x,t)$')
  axs[2,2].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[2,2])

  plt.savefig(f"exp_{exp_name}/raw.png")
  plt.show()

if weights_transfer:
  raw_U, raw_V, _, _ = raw_model.predict(X_star)
  raw_U = raw_U.reshape(t_parts, x_parts)
  raw_V = raw_V.reshape(t_parts, x_parts)
  raw_Q_abs = (raw_U**2 + raw_V**2)**0.5
  U_truth = Exact_u
  V_truth = Exact_v
  Q_abs_truth = (U_truth**2 + V_truth**2)**0.5


  fig, axs = plt.subplots(3, 3, figsize=(21,10), dpi=300)

  for ax in axs.flat:
      ax.set(xlabel='$t$', ylabel='$x$')

  for ax in axs.flat:
      ax.label_outer()

  q_abs_min, q_abs_max = 0, np.abs(Q_abs_truth).max()
  c = axs[0,0].pcolormesh(T, X, Q_abs_truth, cmap='BuPu', vmin=q_abs_min, vmax=q_abs_max)
  axs[0,0].set_title('$|q_{truth}|(x,t)$')
  axs[0,0].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[0,0])

  u_min, u_max = -np.abs(U_truth).max(), np.abs(U_truth).max()
  c = axs[0,1].pcolormesh(T, X, U_truth, cmap='RdBu', vmin=u_min, vmax=u_max)
  axs[0,1].set_title('$u_{truth}(x,t)$')
  axs[0,1].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[0,1])

  c = axs[0,2].pcolormesh(T, X, V_truth, cmap='RdBu', vmin=u_min, vmax=u_max)
  axs[0,2].set_title('$v_{truth}(x,t)$')
  axs[0,2].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[0,2])


  #q_abs_min, q_abs_max = 0, np.abs(Q_abs_calc).max()
  c = axs[1,0].pcolormesh(T, X, raw_Q_abs, cmap='BuPu', vmin=q_abs_min, vmax=q_abs_max)
  axs[1,0].set_title('$|q_{pred}|(x,t)$')
  axs[1,0].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[1,0])

  #u_min, u_max = -np.abs(U_calc).max(), np.abs(U_calc).max()
  c = axs[1,1].pcolormesh(T, X, raw_U, cmap='RdBu', vmin=u_min, vmax=u_max)
  axs[1,1].set_title('$u_{pred}(x,t)$')
  axs[1,1].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[1,1])

  c = axs[1,2].pcolormesh(T, X, raw_V, cmap='RdBu', vmin=u_min, vmax=u_max)
  axs[1,2].set_title('$v_{pred}(x,t)$')
  axs[1,2].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[1,2])


  U_diff = np.abs(U_truth-raw_U)
  V_diff = np.abs(V_truth-raw_V)
  Q_abs_diff = np.abs(Q_abs_truth-raw_Q_abs)

  q_abs_min, q_abs_max = 0, np.abs(Q_abs_diff).max()
  c = axs[2,0].pcolormesh(T, X, Q_abs_diff, cmap='Reds', vmin=q_abs_min, vmax=q_abs_max)
  axs[2,0].set_title('$||q_{truth}|-|q_{pred}||(x,t)$')
  axs[2,0].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[2,0])

  u_min, u_max = U_diff.min(), U_diff.max()
  c = axs[2,1].pcolormesh(T, X, U_diff, cmap='Reds', vmin=u_min, vmax=u_max)
  axs[2,1].set_title('$|u_{truth}-u_{pred}|(x,t)$')
  axs[2,1].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[2,1])

  c = axs[2,2].pcolormesh(T, X, V_diff, cmap='Reds', vmin=u_min, vmax=u_max)
  axs[2,2].set_title('$|v_{truth}-v_{pred}|(x,t)$')
  axs[2,2].axis([t_0, t_1, x_0, x_1])
  fig.colorbar(c, ax=axs[2,2])

  plt.savefig(f"exp_{exp_name}/raw.png")
  plt.show()

In [ ]:
if weights_transfer or values_transfer:
  mse_q = np.mean((Q_abs_truth.flatten() - raw_Q_abs.flatten())**2) #средний квадрат разности модулей
  rel_h = np.linalg.norm(Q_abs_truth.flatten() - raw_Q_abs.flatten(), 2)/np.linalg.norm(Q_abs_truth.flatten(), 2) #отношение нормы разности модулей к норме модуля, принимает значения от 0 до 1, чем ближе к 0 тем лучше
  print(f'MSE_q: {mse_q:.3e}, Rel_h: {rel_h:.3e}')
  stats_file.write(f'Result of source data -- MSE_q: {mse_q:.5e}, Rel_h: {rel_h:.5e}\n')

In [ ]:
%%time
#штука сверху выведет итоговое время выполнения
if basic_learning:
  model = PhysicsInformedNN(X_f_train, _, _, X_uv_train, u_train, v_train, layers, alpha, beta)
if weights_transfer:
  model = PhysicsInformedNN(X_f_train, _, _, X_uv_train, u_train, v_train, layers, alpha, beta)
  model.dnn.load_state_dict(raw_model.dnn.state_dict())
if values_transfer:
  raw_XT = np.hstack([raw_X.flatten()[:,None],raw_T.flatten()[:,None]])
  raw_U = raw_U.flatten()[:,None]
  raw_V = raw_V.flatten()[:,None]
  #idx = np.random.choice(raw_XT.shape[0], N_eq, replace=False)
  #raw_XT = raw_XT[idx,:]
  #raw_U = (raw_U.flatten()[:,None])[idx,:]
  #raw_V = (raw_V.flatten()[:,None])[idx,:]
  model = PhysicsInformedNN(raw_XT, raw_U, raw_V, X_uv_train, u_train, v_train, layers, alpha, beta)

model.train()

In [ ]:
plt.rcParams['figure.dpi'] = 100
plt.plot(np.array(iter_array), np.array(loss_array))
plt.ylim(0,5e-6)
plt.title('loss(iter)')
plt.show()

In [ ]:
torch.save(model, f'model_{exp_name}.pth') #сохраняем модель

In [ ]:
#model = torch.load(f'model_BL_2.pth', map_location=device) #загружаем модель

In [ ]:
try:
    u_pred, v_pred, f_u_pred, f_v_pred = model.predict(X_star)
except:
    print('Data was separated')
    batch_size = 10000
    batches = X_star.shape[0]//batch_size
    u_pred=np.zeros(0)
    v_pred=np.zeros(0)
    for i in range(0, batches):
        if i != batches:
            u_batch, v_batch, f_u_batch, f_u_batch = model.predict(X_star[i*batch_size:(i+1)*batch_size,:])
        else:
            u_batch, v_batch, f_u_batch, f_u_batch = model.predict(X_star[i*batch_size:-1,:])
        u_pred = np.append(u_pred, u_batch)  
        v_pred = np.append(v_pred, v_batch)
    u_pred = np.expand_dims(u_pred, 1)
    v_pred = np.expand_dims(v_pred, 1)

In [ ]:
final_U = u_pred.reshape(t_parts, x_parts)
final_V = v_pred.reshape(t_parts, x_parts)
final_Q_abs = (final_U**2 + final_V**2)**0.5

Q_truth=q(X,T)[0]+1j*q(X,T)[1]
U_truth=np.real(Q_truth)
V_truth=np.imag(Q_truth)
Q_abs_truth=np.abs(Q_truth)

fig, axs = plt.subplots(3, 3, figsize=(21,10), dpi=300)

for ax in axs.flat:
    ax.set(xlabel='$t$', ylabel='$x$')

for ax in axs.flat:
    ax.label_outer()

q_abs_min, q_abs_max = 0, np.abs(Q_abs_truth).max()
c = axs[0,0].pcolormesh(T, X, Q_abs_truth, cmap='BuPu', vmin=q_abs_min, vmax=q_abs_max)
axs[0,0].set_title('$|q_{truth}|(x,t)$')
axs[0,0].axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=axs[0,0])

u_min, u_max = -np.abs(U_truth).max(), np.abs(U_truth).max()
c = axs[0,1].pcolormesh(T, X, U_truth, cmap='RdBu', vmin=u_min, vmax=u_max)
axs[0,1].set_title('$u_{truth}(x,t)$')
axs[0,1].axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=axs[0,1])

c = axs[0,2].pcolormesh(T, X, V_truth, cmap='RdBu', vmin=u_min, vmax=u_max)
axs[0,2].set_title('$v_{truth}(x,t)$')
axs[0,2].axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=axs[0,2])


#q_abs_min, q_abs_max = 0, np.abs(Q_abs_calc).max()
c = axs[1,0].pcolormesh(T, X, final_Q_abs, cmap='BuPu', vmin=q_abs_min, vmax=q_abs_max)
axs[1,0].set_title('$|q_{pred}|(x,t)$')
axs[1,0].axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=axs[1,0])

#u_min, u_max = -np.abs(U_calc).max(), np.abs(U_calc).max()
c = axs[1,1].pcolormesh(T, X, final_U, cmap='RdBu', vmin=u_min, vmax=u_max)
axs[1,1].set_title('$u_{pred}(x,t)$')
axs[1,1].axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=axs[1,1])

c = axs[1,2].pcolormesh(T, X, final_V, cmap='RdBu', vmin=u_min, vmax=u_max)
axs[1,2].set_title('$v_{pred}(x,t)$')
axs[1,2].axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=axs[1,2])


U_diff = np.abs(U_truth-final_U)
V_diff = np.abs(V_truth-final_V)
Q_abs_diff = np.abs(Q_abs_truth-final_Q_abs)

q_abs_min, q_abs_max = 0, np.abs(Q_abs_diff).max()
c = axs[2,0].pcolormesh(T, X, Q_abs_diff, cmap='Reds', vmin=q_abs_min, vmax=q_abs_max)
axs[2,0].set_title('$||q_{truth}|-|q_{pred}||(x,t)$')
axs[2,0].axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=axs[2,0])

u_min, u_max = U_diff.min(), U_diff.max()
c = axs[2,1].pcolormesh(T, X, U_diff, cmap='Reds', vmin=u_min, vmax=u_max)
axs[2,1].set_title('$|u_{truth}-u_{pred}|(x,t)$')
axs[2,1].axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=axs[2,1])

c = axs[2,2].pcolormesh(T, X, V_diff, cmap='Reds', vmin=u_min, vmax=u_max)
axs[2,2].set_title('$|v_{truth}-v_{pred}|(x,t)$')
axs[2,2].axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=axs[2,2])

plt.savefig(f"exp_{exp_name}/enhanced.png")
plt.show()

# Результаты

In [ ]:
mse_q = np.mean((Q_abs_truth.flatten() - final_Q_abs.flatten())**2) #средний квадрат разности модулей
rel_h = np.linalg.norm(Q_abs_truth.flatten() - final_Q_abs.flatten(), 2)/np.linalg.norm(Q_abs_truth.flatten(), 2)
print(f'MSE_q: {mse_q:.3e}, Rel_h: {rel_h:.3e}')
stats_file.write(f'Final result -- MSE_q: {mse_q:.5e}, Rel_h: {rel_h:.5e}\n')
stats_file.close()

In [ ]:
#сохраняем результаты в таблицу
x_star = X.flatten()[:,None]
t_star = T.flatten()[:,None]
u_star = U_truth.flatten()[:,None]
v_star = V_truth.flatten()[:,None]
q_abs_star = Q_abs_truth.flatten()[:,None]
q_abs_pred = (u_pred**2 + v_pred**2)**0.5

data = pd.DataFrame({'x': x_star[:,0],
                     't': t_star[:,0],
                     'true_u': u_star[:,0],
                     'true_v': v_star[:,0],
                     'true_h': q_abs_star[:,0],
                     'pred_u': u_pred[:,0],
                     'pred_v': v_pred[:,0],
                     'pred_h': q_abs_pred[:,0]})
data.to_csv((f"data_{exp_name}.csv"), index=False)